In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Resize, Compose, v2
from torchvision.datasets import Cityscapes
from torch.utils.data import random_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import utils
import wandb



In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenbbs. Use `wandb login --relogin` to force relogin


True

In [3]:
lr_rate = 1e-3
epochs = 10
batch_size = 64
subsize = (128, 256)
momentum = 0.9

In [4]:
augmented_transform = transforms.Compose([
    v2.Resize(subsize),
    v2.RandomHorizontalFlip(1),  # Random horizontal flip
    v2.RandomRotation(10),  # Random rotation up to 10 degrees
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jitter
    v2.ToTensor()
])

augmented_target_transform = transforms.Compose([
    v2.Resize(subsize),
    v2.RandomHorizontalFlip(1),  # Same transformation as input images
    v2.RandomRotation(10),  # Same transformation as input images
    v2.ToTensor()
])

normal_transform = transforms.Compose([
    v2.Resize(subsize),
    v2.ToTensor()
])
normal_target_transform = transforms.Compose([
    v2.Resize(subsize),
    v2.ToTensor()
])


#Desktop
normal_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=normal_transform, target_transform=normal_target_transform)
augmented_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=augmented_transform, target_transform=augmented_target_transform)
dataset = torch.utils.data.ConcatDataset([normal_dataset, augmented_dataset])


#Laptop
#dataset = Cityscapes(root="C:/Users/20182573/Documents/CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)

#subset_small, subset_big = random_split(dataset, [0.2,0.8])
train_dataset, val_dataset = random_split(dataset, [0.8,0.2])

<>:27: SyntaxWarning: invalid escape sequence '\C'
<>:28: SyntaxWarning: invalid escape sequence '\C'
<>:27: SyntaxWarning: invalid escape sequence '\C'
<>:28: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Sven\AppData\Local\Temp\ipykernel_11740\3308899275.py:27: SyntaxWarning: invalid escape sequence '\C'
  normal_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=normal_transform, target_transform=normal_target_transform)
C:\Users\Sven\AppData\Local\Temp\ipykernel_11740\3308899275.py:28: SyntaxWarning: invalid escape sequence '\C'
  augmented_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=augmented_transform, target_transform=augmented_target_transform)
c:\Users\Sven\anaconda3\envs\torch\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.

In [5]:
len(train_dataset)

4760

In [6]:


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [7]:

from model import Model

model = Model().cuda()
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate, momentum=momentum)


In [8]:
def train(dataloader, model, loss_fn, optimizer, run):
    """
    Train a model for 1 epoch.

    Params:
    - dataloader:   dataset to train on.
    - model:        the model object to be trained.
    - loss_fn:      the loss function.
    - optimizer:    the desired optimization.
    """
    size = len(dataloader.dataset)
    model.train() #Set the model to train mode
    for batch, (IMG,SEGM) in enumerate(dataloader):
        IMG = IMG.to('cuda')
        SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
        SEGM = utils.map_id_to_train_id(SEGM).to('cuda')
        
        #predict
        pred = model(IMG)
        #Loss
        loss = loss_fn(pred, SEGM)
        

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print loss during training
        loss, current = loss.item(), (batch + 1) * len(IMG)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        run.log({"train_loss": loss})
    
    return run
            


In [9]:
def test(dataloader, model, loss_fn):
    """
    Test a model.

    Params:
    - dataloader:   dataset to test on.
    - model:        the model object to be tested.
    - loss_fn:      the loss function.
    """
    num_batches = len(dataloader)
    model.eval() #model in eval mode
    test_loss = 0
    with torch.no_grad():
        for _, (IMG,SEGM) in enumerate(dataloader):
            IMG = IMG.to('cuda')
            SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
            SEGM = utils.map_id_to_train_id(SEGM).to('cuda')

            pred = model(IMG)
            test_loss += loss_fn(pred, SEGM).item()
            
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss
    


In [10]:



run = wandb.init(
    # Set the project where this run will be logged
    project="CS_challenge", name="1.2",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": {lr_rate},
        "epochs": {epochs},
        "Momentum": {momentum},
        "Batch_size": {batch_size},
        "model version": 0.0,
        "subset size [%]": 100,
        "resize": {subsize},
    },
)




for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    run = train(train_dataloader, model, loss_fn, optimizer, run)
    test_loss = test(val_dataloader, model, loss_fn)
    run.log({"Test_loss": test_loss})
print("Done!")

run.finish()

Epoch 1
-------------------------------
loss: 3.508835  [   64/ 4760]
loss: 3.502861  [  128/ 4760]
loss: 3.481373  [  192/ 4760]
loss: 3.464029  [  256/ 4760]
loss: 3.432542  [  320/ 4760]
loss: 3.381169  [  384/ 4760]
loss: 3.361730  [  448/ 4760]
loss: 3.311038  [  512/ 4760]
loss: 3.277574  [  576/ 4760]
loss: 3.214880  [  640/ 4760]
loss: 3.213984  [  704/ 4760]
loss: 3.159724  [  768/ 4760]
loss: 3.138999  [  832/ 4760]
loss: 3.126931  [  896/ 4760]
loss: 3.088607  [  960/ 4760]
loss: 3.024116  [ 1024/ 4760]
loss: 3.001950  [ 1088/ 4760]
loss: 2.954829  [ 1152/ 4760]
loss: 2.960425  [ 1216/ 4760]
loss: 2.922375  [ 1280/ 4760]
loss: 2.918529  [ 1344/ 4760]
loss: 2.866815  [ 1408/ 4760]
loss: 2.823286  [ 1472/ 4760]
loss: 2.837028  [ 1536/ 4760]
loss: 2.752928  [ 1600/ 4760]
loss: 2.808018  [ 1664/ 4760]
loss: 2.714179  [ 1728/ 4760]
loss: 2.728692  [ 1792/ 4760]
loss: 2.682433  [ 1856/ 4760]
loss: 2.651441  [ 1920/ 4760]
loss: 2.654855  [ 1984/ 4760]
loss: 2.631432  [ 2048/ 4760]


Test_loss,█▅▄▃▂▂▁▁▁▁
train_loss,█▇▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_loss,0.95983
train_loss,0.87789


In [11]:
torch.save(model.state_dict(), "model.pth")